### NER (450000 data)

In [1]:
import spacy
from spacy.tokens import DocBin
from spacy.util import filter_spans
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
import json
import difflib
from fuzzywuzzy import fuzz

In [1]:
def preprocess_data(address):
    '''
    Arguments:
        address (String): full address
    Return:
        cleansed_address (String): standardised address form
    '''
    cleansed_address1 = re.sub(r'(,)(?!\s)',', ', address) # add space after comma if there is none
    cleansed_address2 = re.sub(r'(\\n)',', ', cleansed_address1) # converts newline to a comma and space
    cleansed_address3 = re.sub(r'(?!\s)(-)(?!\s)',' - ', cleansed_address2) # adds space around hyphens
    cleansed_address = re.sub(r'\.','', cleansed_address3) # removes period
    return cleansed_address


def get_address_span(address=None, address_component=None, label=None):
    '''
    Retrieve the start index, end index and the corresponding label given the relevant arguments.
    Note for end index: up to but not including
    
    Arguments:
        address (String): full address
        address_component (String): address component to search for in the address string (eg. "123A")
        label: corresponding label for the address component (eg. "BLOCK")
    Return:
        (start_index, end_index, label): a tuple containing the start index, end index and label of address component in the address string (eg. (0,4,"block"))
        Returns None if address component is NaN or if span is not found.
    '''
    if pd.isna(address_component) or str(address_component)=='nan':
        span = None
        pass
    else:                               
        address_component1=re.sub('\.','',address_component)                               
        address_component2=re.sub(r'(?!\s)(-)(?!\s)',' - ',address_component1)                               
        span=re.search('\\b(?:'+address_component2+')\\b',address)                                                              
    if span is None:
         pass
    else:
        #print(address_component)
        #print("(" + str(span.start()) + ", " + str(span.end()) + ", " + label + ")")
        return (span.start(), span.end(), label)
    
    
def extend_list(entity_list, entity):
    '''
    Adds entity to the entity list if there is an entity
    
    Arguments:
        entity_list (List): entity tag list (eg. ["BlockTag","StreetTag","BuildingTag","FloorNumTag","UnitNumTag","PostalCodeTag","CountryTag"])
        entity (String): tag representing an entity (eg. "BlockTag") 
    Returns:
        entity_list: extended entity tag list
    '''
    if pd.isna(entity):
        return entity_list
    else:
        entity_list.append(entity)
        return entity_list
    
    
def create_entity_spans(df, entity_tag_list):
    '''
    Create the entity spans given the full address, labelled components and entity_tag_list
    
    Arguments:
        df (DataFrame): dataframe containing full address and the labelled components
        entity_tag_list (List): entity tag list (eg. ["BlockTag","StreetTag","BuildingTag","FloorNumTag","UnitNumTag","PostalCodeTag","CountryTag"])
    Return:
        df['EntitySpans'] (DataFrame): dataframe consisting of full address and list of entity spans (eg. ('123A ABC STREET ...', [(0,4,'BLOCK'),...]))
    '''
    df['Address']=df['address'].apply(lambda x: preprocess_data(x))
#     df["UnitTag"]=df.apply(lambda row:get_address_span(address=row['address'],address_component=str(row['unit']),label='UNIT'),axis=1)
    df["StreetTag"]=df.apply(lambda row:get_address_span(address=row['address'],address_component=str(row['street']),label='STREET'),axis=1)
    df["ZipcodeTag"]=df.apply(lambda row:get_address_span(address=row['address'],address_component=str(row['zipcode']),label='ZIPCODE'),axis=1)
    df["CityTag"]=df.apply(lambda row:get_address_span(address=row['address'],address_component=str(row['city']),label='CITY'),axis=1)
    df["ProvinceTag"]=df.apply(lambda row:get_address_span(address=row['address'],address_component=str(row['province']),label='PROVINCE'),axis=1)
    df["CountryTag"]=df.apply(lambda row:get_address_span(address=row['address'],address_component=str(row['country']),label='COUNTRY'),axis=1)
    
    df['EmptySpan']=df.apply(lambda x: [], axis=1)

    for i in entity_tag_list:
        df['EntitySpans']=df.apply(lambda row: extend_list(row['EmptySpan'],row[i]),axis=1)
        df['EntitySpans']=df[['EntitySpans','address']].apply(lambda x: (x[1], x[0]),axis=1)
    return df['EntitySpans']


def get_doc_bin(training_data,nlp):
    '''
    Create DocBin object for building training/test corpus
    
    Arguments:
        training_data (DataFrame): training data from train test split
        nlp (spacy.Language): instance of the language model, can be initialised to "en" for English
    Return:
        db: DocBin object prepared for training model
    '''
    # the DocBin will store the example documents
 
    db = DocBin()
    for text, annotations in training_data:
        doc = nlp(text) #Construct a Doc object
        ents = []
        for start, end, label in annotations:
            span = doc.char_span(start, end, label=label)
            ents.append(span)
        try:
            filtered = filter_spans(ents)
            doc.ents = filtered
        except:
            doc.ents = []
        #doc.ents = ents
        #print(doc)
        db.add(doc)
    return db

In [3]:
entity_tag_list = ["StreetTag", "ZipcodeTag", "CityTag", "ProvinceTag", "CountryTag"]

### Load dataset

In [81]:
df = pd.read_pickle("./data/data_collection/foreign_combined_w_labels.pkl")

In [10]:
train, test_data = train_test_split(df, test_size = 0.2)

In [11]:
train_data, validation_data = train_test_split(train, test_size = 0.2)

In [86]:
train_data.to_pickle("./data/train/train_data.pkl")
train_data.to_csv("./data/train/train_data.csv", index=False)
validation_data.to_pickle("./data/train/validation_data.pkl")
validation_data.to_csv("./data/train/validation_data.csv", index=False)

test_data.to_pickle("./data/test/test_data.pkl")
test_data.to_csv("./data/test/test_data.csv", index=False)

In [13]:
nlp = spacy.blank("id")

### Preparation for training dataset

In [14]:
df_entity_spans = create_entity_spans(train_data.astype(str), entity_tag_list)

In [15]:
training_data = df_entity_spans.values.tolist()

In [16]:
doc_bin_train = get_doc_bin(training_data, nlp)

In [17]:
# serialize the DocBin object and save the training data in a binary format suitable for NER model training
doc_bin_train.to_disk("./corpus/spacy-docbins/train.spacy")

### Preparation for validation dataset

In [18]:
df_entity_spans_validation = create_entity_spans(validation_data.astype(str), entity_tag_list)

In [19]:
validating_data = df_entity_spans_validation.values.tolist()

In [20]:
doc_bin_validation = get_doc_bin(validating_data, nlp)

In [21]:
doc_bin_validation.to_disk("./corpus/spacy-docbins/test.spacy")

### Predict fields for test data

In [22]:
def predict_fields(df):
    '''
    Predict the address components given a dataframe containing full addresses
    
    Arguments:
        df (DataFrame): a dataframe containing a column of addresses
    Return:
        tested_data (DataFrame): a dataframe containing the orignal addresses with the predicted address components 
    '''
    house_list = []
    street_list = []
    zipcode_list = []
    city_list = []
    province_list = []
    country_list = []

    for index, address in enumerate(df['address']):

        # note: ent.label is an integer identifier for entity while ent.label_ uses a string representation
        entities = {ent.label_: ent.text for ent in nlp(str(address)).ents}

#         house_list.append(entities.get('HOUSE', ''))
        street_list.append(entities.get('STREET', ''))
        zipcode_list.append(entities.get('ZIPCODE', ''))
        city_list.append(entities.get('CITY', ''))
        province_list.append(entities.get('PROVINCE', ''))
        country_list.append(entities.get('COUNTRY', ''))

        print(f"Iteration: {index+1}/{len(df)}")

    tested_data = df.assign(
#         predicted_house = house_list,
        predicted_street = street_list,
        predicted_zipcode = zipcode_list,
        predicted_city = city_list,
        predicted_province = province_list,
        predicted_country = country_list
    )
    
    tested_data = tested_data.replace(r'^\s*$', np.nan, regex=True)
    
    return tested_data

In [23]:
def calculate_similarity(actual, predicted, threshold):
    '''
    Decides if predicted is similar to actual based on a set threshold.
    
    Arguments:
        actual (DataFrame): dataframe containing the labelled components
        predicted (DataFrame): dataframe containing the predicted components
        threshold (Float): a minimum value for predicted field to be considered similar to the labelled field
    Return:
        0 or 1 (Integer): 0 if not similar, 1 if similar
    '''
    actual_str = str(actual)
    predicted_str = str(predicted)
    similarity = difflib.SequenceMatcher(None, actual_str, predicted_str).ratio()
    
    # Check if the threshold is a dictionary or a scalar value
    if isinstance(threshold, dict):
        field_threshold = threshold.get(actual)
        if field_threshold is not None and similarity >= field_threshold:
            return 1
    else:
        if similarity >= threshold:
            return 1
    
    return 0

def calculate_accuracy(tested_df, thresholds):
    '''
    Compares the predicted components with the originally labelled components and calculates an accuracy score
    
    Arguments:
        df (DataFrame): dataframe containing full address, originally labelled components and the predicted components
    Return:
        tested_df (DataFrame): dataframe containing an addition column with accuracy score
    '''
    accuracy_values = []
    for _, row in tested_df.iterrows():
        actual_fields = [row[f'{field}'] for field in ['street', 'zipcode', 'city', 'province', 'country']]
        predicted_fields = [row[f'predicted_{field}'] for field in ['street', 'zipcode', 'city', 'province', 'country']]
        matches = []
        for actual, predicted, field in zip(actual_fields, predicted_fields, ['street', 'zipcode', 'city', 'province', 'country']):
            threshold = thresholds.get(field, 0.8)  # Use a default threshold of 0.8 if not specified for the field
            matches.append(calculate_similarity(actual, predicted, threshold))
        accuracy = sum(matches) / len(matches)
        accuracy_values.append(accuracy)
    tested_df['accuracy'] = accuracy_values
    return tested_df

### Adding rule-based approach to NER

In [26]:
# Retrieving unique cities, provinces and countries
unique_cities = train_data['city'].unique()
unique_provinces = train_data['province'].unique()
unique_countries = train_data['country'].unique()

city_patterns = [{"label": "CITY", "pattern": str(city)} for city in unique_cities]
province_patterns = [{"label": "PROVINCE", "pattern": str(province)} for province in unique_provinces]
country_patterns = [{"label": "COUNTRY", "pattern": str(country)} for country in unique_countries]

field_patterns =  city_patterns + province_patterns + country_patterns

In [27]:
file_path = "corpus/rules/entity_ruler_patterns.jsonl"

# Write the field_patterns list to the JSON file
with open(file_path, "w") as jsonl_file:
    for field_pattern in field_patterns:
        jsonl_file.write(json.dumps(field_pattern) + "\n")

print("Entity rule patterns saved to:", file_path)

Entity rule patterns saved to: corpus/rules/entity_ruler_patterns.jsonl


In [23]:
# ML NER (better performance since addresses are not as structured, more spelling variations etc)
# ML + ER NER

### Input address for testing

In [28]:
nlp = spacy.load("output/models/model-best")

In [37]:
address_list = [
    "NO. 44 JALAN DESA MELUR 56000 CHERAS KUALA LUMPUR MALAYSIA",
    "46 JALAN RUBY IV DKI JAKARTA JARKARTA 11610 INDONESIA"
]

In [38]:
for address in address_list:
    doc=nlp(address)
    print('doc: '+ str(doc))
    ent_list=[(ent.text, ent.label_) for ent in doc.ents]
    #print('ent list: ' + str(ent_list))
    print("Address string -> "+address)
    print("Parsed address -> "+str(ent_list))
    print("******")

doc: NO. 44 JALAN DESA MELUR 56000 CHERAS KUALA LUMPUR MALAYSIA
Address string -> NO. 44 JALAN DESA MELUR 56000 CHERAS KUALA LUMPUR MALAYSIA
Parsed address -> [('NO. 44 JALAN DESA MELUR', 'STREET'), ('56000', 'ZIPCODE'), ('CHERAS', 'CITY'), ('KUALA LUMPUR', 'PROVINCE'), ('MALAYSIA', 'COUNTRY')]
******
doc: 46 JALAN RUBY IV DKI JAKARTA 11610 INDONESIA
Address string -> 46 JALAN RUBY IV DKI JAKARTA 11610 INDONESIA
Parsed address -> [('46 JALAN RUBY IV', 'STREET'), ('DKI JAKARTA', 'PROVINCE'), ('11610', 'ZIPCODE'), ('INDONESIA', 'COUNTRY')]
******


### Run test data through the trained model (ML NER)

In [39]:
nlp = spacy.load("output\models\model-best")

In [87]:
test_data = pd.read_pickle('./data/test/test_data.pkl')

In [88]:
tested = predict_fields(test_data)

Iteration: 1/89557
Iteration: 2/89557
Iteration: 3/89557
Iteration: 4/89557
Iteration: 5/89557
Iteration: 6/89557
Iteration: 7/89557
Iteration: 8/89557
Iteration: 9/89557
Iteration: 10/89557
Iteration: 11/89557
Iteration: 12/89557
Iteration: 13/89557
Iteration: 14/89557
Iteration: 15/89557
Iteration: 16/89557
Iteration: 17/89557
Iteration: 18/89557
Iteration: 19/89557
Iteration: 20/89557
Iteration: 21/89557
Iteration: 22/89557
Iteration: 23/89557
Iteration: 24/89557
Iteration: 25/89557
Iteration: 26/89557
Iteration: 27/89557
Iteration: 28/89557
Iteration: 29/89557
Iteration: 30/89557
Iteration: 31/89557
Iteration: 32/89557
Iteration: 33/89557
Iteration: 34/89557
Iteration: 35/89557
Iteration: 36/89557
Iteration: 37/89557
Iteration: 38/89557
Iteration: 39/89557
Iteration: 40/89557
Iteration: 41/89557
Iteration: 42/89557
Iteration: 43/89557
Iteration: 44/89557
Iteration: 45/89557
Iteration: 46/89557
Iteration: 47/89557
Iteration: 48/89557
Iteration: 49/89557
Iteration: 50/89557
Iteration

Iteration: 413/89557
Iteration: 414/89557
Iteration: 415/89557
Iteration: 416/89557
Iteration: 417/89557
Iteration: 418/89557
Iteration: 419/89557
Iteration: 420/89557
Iteration: 421/89557
Iteration: 422/89557
Iteration: 423/89557
Iteration: 424/89557
Iteration: 425/89557
Iteration: 426/89557
Iteration: 427/89557
Iteration: 428/89557
Iteration: 429/89557
Iteration: 430/89557
Iteration: 431/89557
Iteration: 432/89557
Iteration: 433/89557
Iteration: 434/89557
Iteration: 435/89557
Iteration: 436/89557
Iteration: 437/89557
Iteration: 438/89557
Iteration: 439/89557
Iteration: 440/89557
Iteration: 441/89557
Iteration: 442/89557
Iteration: 443/89557
Iteration: 444/89557
Iteration: 445/89557
Iteration: 446/89557
Iteration: 447/89557
Iteration: 448/89557
Iteration: 449/89557
Iteration: 450/89557
Iteration: 451/89557
Iteration: 452/89557
Iteration: 453/89557
Iteration: 454/89557
Iteration: 455/89557
Iteration: 456/89557
Iteration: 457/89557
Iteration: 458/89557
Iteration: 459/89557
Iteration: 46

Iteration: 836/89557
Iteration: 837/89557
Iteration: 838/89557
Iteration: 839/89557
Iteration: 840/89557
Iteration: 841/89557
Iteration: 842/89557
Iteration: 843/89557
Iteration: 844/89557
Iteration: 845/89557
Iteration: 846/89557
Iteration: 847/89557
Iteration: 848/89557
Iteration: 849/89557
Iteration: 850/89557
Iteration: 851/89557
Iteration: 852/89557
Iteration: 853/89557
Iteration: 854/89557
Iteration: 855/89557
Iteration: 856/89557
Iteration: 857/89557
Iteration: 858/89557
Iteration: 859/89557
Iteration: 860/89557
Iteration: 861/89557
Iteration: 862/89557
Iteration: 863/89557
Iteration: 864/89557
Iteration: 865/89557
Iteration: 866/89557
Iteration: 867/89557
Iteration: 868/89557
Iteration: 869/89557
Iteration: 870/89557
Iteration: 871/89557
Iteration: 872/89557
Iteration: 873/89557
Iteration: 874/89557
Iteration: 875/89557
Iteration: 876/89557
Iteration: 877/89557
Iteration: 878/89557
Iteration: 879/89557
Iteration: 880/89557
Iteration: 881/89557
Iteration: 882/89557
Iteration: 88

Iteration: 1240/89557
Iteration: 1241/89557
Iteration: 1242/89557
Iteration: 1243/89557
Iteration: 1244/89557
Iteration: 1245/89557
Iteration: 1246/89557
Iteration: 1247/89557
Iteration: 1248/89557
Iteration: 1249/89557
Iteration: 1250/89557
Iteration: 1251/89557
Iteration: 1252/89557
Iteration: 1253/89557
Iteration: 1254/89557
Iteration: 1255/89557
Iteration: 1256/89557
Iteration: 1257/89557
Iteration: 1258/89557
Iteration: 1259/89557
Iteration: 1260/89557
Iteration: 1261/89557
Iteration: 1262/89557
Iteration: 1263/89557
Iteration: 1264/89557
Iteration: 1265/89557
Iteration: 1266/89557
Iteration: 1267/89557
Iteration: 1268/89557
Iteration: 1269/89557
Iteration: 1270/89557
Iteration: 1271/89557
Iteration: 1272/89557
Iteration: 1273/89557
Iteration: 1274/89557
Iteration: 1275/89557
Iteration: 1276/89557
Iteration: 1277/89557
Iteration: 1278/89557
Iteration: 1279/89557
Iteration: 1280/89557
Iteration: 1281/89557
Iteration: 1282/89557
Iteration: 1283/89557
Iteration: 1284/89557
Iteration:

Iteration: 1661/89557
Iteration: 1662/89557
Iteration: 1663/89557
Iteration: 1664/89557
Iteration: 1665/89557
Iteration: 1666/89557
Iteration: 1667/89557
Iteration: 1668/89557
Iteration: 1669/89557
Iteration: 1670/89557
Iteration: 1671/89557
Iteration: 1672/89557
Iteration: 1673/89557
Iteration: 1674/89557
Iteration: 1675/89557
Iteration: 1676/89557
Iteration: 1677/89557
Iteration: 1678/89557
Iteration: 1679/89557
Iteration: 1680/89557
Iteration: 1681/89557
Iteration: 1682/89557
Iteration: 1683/89557
Iteration: 1684/89557
Iteration: 1685/89557
Iteration: 1686/89557
Iteration: 1687/89557
Iteration: 1688/89557
Iteration: 1689/89557
Iteration: 1690/89557
Iteration: 1691/89557
Iteration: 1692/89557
Iteration: 1693/89557
Iteration: 1694/89557
Iteration: 1695/89557
Iteration: 1696/89557
Iteration: 1697/89557
Iteration: 1698/89557
Iteration: 1699/89557
Iteration: 1700/89557
Iteration: 1701/89557
Iteration: 1702/89557
Iteration: 1703/89557
Iteration: 1704/89557
Iteration: 1705/89557
Iteration:

...

Iteration: 87847/89557
Iteration: 87848/89557
Iteration: 87849/89557
Iteration: 87850/89557
Iteration: 87851/89557
Iteration: 87852/89557
Iteration: 87853/89557
Iteration: 87854/89557
Iteration: 87855/89557
Iteration: 87856/89557
Iteration: 87857/89557
Iteration: 87858/89557
Iteration: 87859/89557
Iteration: 87860/89557
Iteration: 87861/89557
Iteration: 87862/89557
Iteration: 87863/89557
Iteration: 87864/89557
Iteration: 87865/89557
Iteration: 87866/89557
Iteration: 87867/89557
Iteration: 87868/89557
Iteration: 87869/89557
Iteration: 87870/89557
Iteration: 87871/89557
Iteration: 87872/89557
Iteration: 87873/89557
Iteration: 87874/89557
Iteration: 87875/89557
Iteration: 87876/89557
Iteration: 87877/89557
Iteration: 87878/89557
Iteration: 87879/89557
Iteration: 87880/89557
Iteration: 87881/89557
Iteration: 87882/89557
Iteration: 87883/89557
Iteration: 87884/89557
Iteration: 87885/89557
Iteration: 87886/89557
Iteration: 87887/89557
Iteration: 87888/89557
Iteration: 87889/89557
Iteration: 

Iteration: 88281/89557
Iteration: 88282/89557
Iteration: 88283/89557
Iteration: 88284/89557
Iteration: 88285/89557
Iteration: 88286/89557
Iteration: 88287/89557
Iteration: 88288/89557
Iteration: 88289/89557
Iteration: 88290/89557
Iteration: 88291/89557
Iteration: 88292/89557
Iteration: 88293/89557
Iteration: 88294/89557
Iteration: 88295/89557
Iteration: 88296/89557
Iteration: 88297/89557
Iteration: 88298/89557
Iteration: 88299/89557
Iteration: 88300/89557
Iteration: 88301/89557
Iteration: 88302/89557
Iteration: 88303/89557
Iteration: 88304/89557
Iteration: 88305/89557
Iteration: 88306/89557
Iteration: 88307/89557
Iteration: 88308/89557
Iteration: 88309/89557
Iteration: 88310/89557
Iteration: 88311/89557
Iteration: 88312/89557
Iteration: 88313/89557
Iteration: 88314/89557
Iteration: 88315/89557
Iteration: 88316/89557
Iteration: 88317/89557
Iteration: 88318/89557
Iteration: 88319/89557
Iteration: 88320/89557
Iteration: 88321/89557
Iteration: 88322/89557
Iteration: 88323/89557
Iteration: 

Iteration: 88694/89557
Iteration: 88695/89557
Iteration: 88696/89557
Iteration: 88697/89557
Iteration: 88698/89557
Iteration: 88699/89557
Iteration: 88700/89557
Iteration: 88701/89557
Iteration: 88702/89557
Iteration: 88703/89557
Iteration: 88704/89557
Iteration: 88705/89557
Iteration: 88706/89557
Iteration: 88707/89557
Iteration: 88708/89557
Iteration: 88709/89557
Iteration: 88710/89557
Iteration: 88711/89557
Iteration: 88712/89557
Iteration: 88713/89557
Iteration: 88714/89557
Iteration: 88715/89557
Iteration: 88716/89557
Iteration: 88717/89557
Iteration: 88718/89557
Iteration: 88719/89557
Iteration: 88720/89557
Iteration: 88721/89557
Iteration: 88722/89557
Iteration: 88723/89557
Iteration: 88724/89557
Iteration: 88725/89557
Iteration: 88726/89557
Iteration: 88727/89557
Iteration: 88728/89557
Iteration: 88729/89557
Iteration: 88730/89557
Iteration: 88731/89557
Iteration: 88732/89557
Iteration: 88733/89557
Iteration: 88734/89557
Iteration: 88735/89557
Iteration: 88736/89557
Iteration: 

Iteration: 89094/89557
Iteration: 89095/89557
Iteration: 89096/89557
Iteration: 89097/89557
Iteration: 89098/89557
Iteration: 89099/89557
Iteration: 89100/89557
Iteration: 89101/89557
Iteration: 89102/89557
Iteration: 89103/89557
Iteration: 89104/89557
Iteration: 89105/89557
Iteration: 89106/89557
Iteration: 89107/89557
Iteration: 89108/89557
Iteration: 89109/89557
Iteration: 89110/89557
Iteration: 89111/89557
Iteration: 89112/89557
Iteration: 89113/89557
Iteration: 89114/89557
Iteration: 89115/89557
Iteration: 89116/89557
Iteration: 89117/89557
Iteration: 89118/89557
Iteration: 89119/89557
Iteration: 89120/89557
Iteration: 89121/89557
Iteration: 89122/89557
Iteration: 89123/89557
Iteration: 89124/89557
Iteration: 89125/89557
Iteration: 89126/89557
Iteration: 89127/89557
Iteration: 89128/89557
Iteration: 89129/89557
Iteration: 89130/89557
Iteration: 89131/89557
Iteration: 89132/89557
Iteration: 89133/89557
Iteration: 89134/89557
Iteration: 89135/89557
Iteration: 89136/89557
Iteration: 

Iteration: 89495/89557
Iteration: 89496/89557
Iteration: 89497/89557
Iteration: 89498/89557
Iteration: 89499/89557
Iteration: 89500/89557
Iteration: 89501/89557
Iteration: 89502/89557
Iteration: 89503/89557
Iteration: 89504/89557
Iteration: 89505/89557
Iteration: 89506/89557
Iteration: 89507/89557
Iteration: 89508/89557
Iteration: 89509/89557
Iteration: 89510/89557
Iteration: 89511/89557
Iteration: 89512/89557
Iteration: 89513/89557
Iteration: 89514/89557
Iteration: 89515/89557
Iteration: 89516/89557
Iteration: 89517/89557
Iteration: 89518/89557
Iteration: 89519/89557
Iteration: 89520/89557
Iteration: 89521/89557
Iteration: 89522/89557
Iteration: 89523/89557
Iteration: 89524/89557
Iteration: 89525/89557
Iteration: 89526/89557
Iteration: 89527/89557
Iteration: 89528/89557
Iteration: 89529/89557
Iteration: 89530/89557
Iteration: 89531/89557
Iteration: 89532/89557
Iteration: 89533/89557
Iteration: 89534/89557
Iteration: 89535/89557
Iteration: 89536/89557
Iteration: 89537/89557
Iteration: 

In [89]:
tested = tested[['address', 'predicted_street', 'predicted_zipcode', 'predicted_city', 'predicted_province', 'predicted_country', 'street', 'zipcode', 'city', 'province', 'country']]

thresholds = {
    'street': 0.75,
    'zipcode': 1.0,
    'city': 0.75,
    'province': 0.75,
    'country': 0.75
}

tested_w_accuracy = calculate_accuracy(tested, thresholds)
tested_w_accuracy

,address,predicted_street,predicted_zipcode,predicted_city,predicted_province,predicted_country,street,zipcode,city,province,country,accuracy
363728,PAROTO LILIRILAU (LILI RILAU) NO. 980 SOPPENG ...,PAROTO LILIRILAU (LILI RILAU) NO. 980,90871,SOPPENG,SULAWESI SELATAN,INDONESIA,PAROTO LILIRILAU (LILI RILAU) NO. 980,90871,SOPPENG,SULAWESI SELATAN,INDONESIA,1.0
231334,JL CONDET RAYA 52 D DKI JAKARTA 13520 INDONESIA,JL CONDET RAYA 52 D,13520,NaN,DKI JAKARTA,INDONESIA,JL CONDET RAYA 52 D,13520,NaN,DKI JAKARTA,INDONESIA,1.0
319004,LEPASAN BAKUMPAI 532K BARITO KUALA KALIMANTAN ...,LEPASAN BAKUMPAI 532K,70512,BARITO KUALA,KALIMANTAN SELATAN,INDONESIA,LEPASAN BAKUMPAI 532K,70512,BARITO KUALA,KALIMANTAN SELATAN,INDONESIA,1.0
434523,TEGALPANJANG (TEGAL PANJANG) CARIU NO. 480 BOG...,TEGALPANJANG (TEGAL PANJANG) CARIU NO. 480,16840,BOGOR,JAWA BARAT,INDONESIA,TEGALPANJANG (TEGAL PANJANG) CARIU NO. 480,16840,BOGOR,JAWA BARAT,INDONESIA,1.0
37834,NO. 854 JALAN CANTIK 13400 BUTTERWORTH PULAU P...,NO. 854 JALAN CANTIK,13400,BUTTERWORTH,PULAU PINANG,MALAYSIA,NO. 854 JALAN CANTIK,13400,BUTTERWORTH,PULAU PINANG,MALAYSIA,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
406469,SINGASARI JONGGOL 792H BOGOR JAWA BARAT 16830 ...,SINGASARI JONGGOL 792H,16830,BOGOR,JAWA BARAT,INDONESIA,SINGASARI JONGGOL 792H,16830,BOGOR,JAWA BARAT,INDONESIA,1.0
55814,NO. 141 KAMPUNG BERANGAN MASJID 16200 TUMPAT K...,NO. 141 KAMPUNG BERANGAN MASJID,16200,TUMPAT,KELANTAN,MALAYSIA,NO. 141 KAMPUNG BERANGAN MASJID,16200,TUMPAT,KELANTAN,MALAYSIA,1.0
194218,82 TAMAN SERI SENAI 81400 SENAI JOHOR MALAYSIA,82 TAMAN SERI SENAI,81400,NaN,JOHOR,MALAYSIA,82 TAMAN SERI SENAI,81400,SENAI,JOHOR,MALAYSIA,0.8
148833,152S TAMAN DESA SEGAMBUT 51200 KUALA LUMPUR WP...,152S TAMAN DESA SEGAMBUT,51200,KUALA LUMPUR,WP KUALA LUMPUR,MALAYSIA,152S TAMAN DESA SEGAMBUT,51200,KUALA LUMPUR,WP KUALA LUMPUR,MALAYSIA,1.0


In [90]:
tested_w_accuracy.to_pickle("./data/test/tested.pkl")
tested_w_accuracy.to_csv("./data/test/tested.csv", index=False)